In [1]:
%pylab nbagg
from tvb.simulator.lab import *
from sklearn.ensemble import ExtraTreesRegressor
import operator
from scipy.signal import butter, lfilter
import scipy

import pickle
import copy
import os

Populating the interactive namespace from numpy and matplotlib
   INFO  log level set to INFO


In [2]:
datapath = 'data/'
savepath = 'fqi_test/'
datatime = 10000
len_frame = 100
dt = 1
n_frame = len_frame / dt
ic_buffer = 3000
runtime = 20000
action_weight = 2

learn = 0.2
future = 0.5
default = 0

actions = [('non',),
# ('sin', 1.0, 10, True),
 ('sin', 1.0, 25, True),
 ('sin', 1.0, 5, True),
# ('sin', 1.5, 25, True),
# ('sin', 1.5, 50, True),
# ('sin', 0.4, 25, True),
# ('sin', 0.5, 10, True),
 ('sin', 0.5, 25, True),
# ('sin', 0.5, 50, True),
 ('sin', 0.5, 5, True),
 ('sqr', 1.0, 10, True, 10),
 ('sqr', 1.0, 2, True, 2),
 ('sqr', 1.0, 4, True, 2),
 ('sqr', 1.0, 4, True, 4),
# ('sqr', 1.0, 50, True, 10),
# ('sqr', 1.0, 50, True, 2),
 ('sqr', 1.5, 10, True, 10),
 ('sqr', 1.5, 2, True, 2),
 ('sqr', 1.5, 4, True, 2),
 ('sqr', 1.5, 4, True, 4),
# ('sqr', 1.5, 50, True, 10),
# ('sqr', 1.5, 50, True, 2),

 ('sqr', 2.0, 50, True, 2),
]

In [3]:
learns = [0.1,0.8]
futures = [0.0,0.8]
action_weights = [4.0]
temperatures = [0.25]
defaults = [0.0,-1.0, -100.0, 1.0, 100.0]

In [14]:
learns = [0.4, 0.8]
futures = [0.0, 0.8]
action_weights = [4.0, 2.0, 1.0, 0.5, 0.0]
temperatures = [0.01, 0.1, 0.25, 0.5, 1]
defaults = [-10.0, -1.0, 0.0,-1.0, -10.0, 1.0]

In [ ]:
results = []
ioff()
for tau in temperatures:
    for l in learns:
        for f in futures:
            for a in action_weights:
                for df in defaults:

                   

                    filename = "sweep-d" + str(int(df)) +  "-l" + str(int(100*l)) + "-f" + str(int(100*f)) + "-a" + str(int(100*a)) + "-t" + str(int(100*tau))


                    done = os.listdir(savepath)
                    if (filename + '.p') not in done and (abs(df / tau) < 120.0) :
                        
                        try:

                            print "\nRunning " + filename
                            learn = l
                            future = f
                            action_weight = a
                            temperature = tau
                            default = df

                            (t, d, rew, amp, act, tups) = online_learn()
                            params = (learn, future, action_weight, temperature)
                            (total_seizure_len, average_length, stim_time) = evaluate_run(tups)
                            results.append( ( params, (t, d, rew, amp, act, tups), (total_seizure_len, average_length, stim_time) ) )


                            tit = "Stimulation with parameters: " + "Def: " + str(int(df)) + "Learning Rate: " + str(l) + " Discount: " + str(f) + " Action Penalty: " + str(a) + " Softmax: " + str(tau) + "\n Results: Total Seizure Time: " + str(total_seizure_len) + " Average Seizure Length: " + str(average_length) + " Stim Time: " +str(stim_time)

                            save_plot(d, amp, rew, tit,filename)

                            savefile = open('qparamsweep/' + filename + '.p', 'w')
                            pickle.dump( (params, (total_seizure_len, average_length, stim_time), t, d, tups ) , savefile  )
                            savefile.close()
                            
                        except:
                            print "\n   Run failed for file " + filename
                    else:
                        print "\nSkipping " + filename



Skipping sweep-d-10-l40-f0-a400-t1

Running sweep-d-1-l40-f0-a400-t1
WARNING  File 'hemispheres' not found in ZIP.
  Starting iterations
    Running frame: 1

Running sweep-d0-l40-f0-a400-t1
WARNING  File 'hemispheres' not found in ZIP.
  Starting iterations
    Running frame: 1

Running sweep-d-1-l40-f0-a400-t1
WARNING  File 'hemispheres' not found in ZIP.
  Starting iterations
    Running frame: 1

Skipping sweep-d-10-l40-f0-a400-t1

Running sweep-d1-l40-f0-a400-t1
WARNING  File 'hemispheres' not found in ZIP.
  Starting iterations
    Running frame: 1

Skipping sweep-d-10-l40-f0-a200-t1

Running sweep-d-1-l40-f0-a200-t1
WARNING  File 'hemispheres' not found in ZIP.
  Starting iterations
    Running frame: 1

Running sweep-d0-l40-f0-a200-t1
WARNING  File 'hemispheres' not found in ZIP.


In [4]:
def save_plot(d, amp, rew, tit, filename):
    fig, ax1 = pyplot.subplots(figsize=(15,5))
    ax2 = ax1.twinx()

    ax1.set_ylim(-3, 3)
    ax1.set_ylabel('LFP')
    ax2.set_ylim(-2, 2)
    ax2.set_ylabel('Stimulus')

    ax2.plot(amp, 'b', alpha = 0.3)
    ax1.plot(d[ic_buffer:,0,9,0], 'r')
    ax2.plot(len_frame * np.array(range(len(rew))), rew, 'g')

    red = matplotlib.patches.Patch(color='red', label = 'LFP')
    blue = matplotlib.patches.Patch(color='blue', label = 'Stimulus')
    legend(handles=[red,blue])
    
    title(tit)
    
    fig.savefig("qparamsweep/" + filename)
    
    #show()

In [5]:
def evaluate_run(tuples):
    icts = []
    past_ict = 0
    ict_len = 0
    ict_lens = []
    stim_time = 0
    
    for tup in tuples:
        
        if tup[1] != ('non',):
            stim_time += 1
        
        if tup[0] == (-1,):
            ict = 1
            icts.append(1)
            ict_len += 1
        else:
            ict = 0
            icts.append(0)
        if past_ict == 1 and ict == 0:
            ict_lens.append(ict_len)
            ict_len = 0
        past_ict = ict
    if ict != 0:
        ict_lens.append(ict_len)
        
    total_seizure_len = sum(icts)
    average_length = float(sum(ict_lens)) / float(len(ict_lens))
    
    return (total_seizure_len, average_length, stim_time)
    
    
    
    

In [6]:
def online_learn():
    
    t_prep, d_prep, sim_test, stim_test = prepare_sim()
    
    actions_all = []
    amps_all = []
    d_all = d_prep
    t_all = t_prep
    rewards_all = []
    prevlfp = 0
    states_all = []
    tuples_all = []
    q_table = {}

    print "  Starting iterations"
    sys.stdout.flush()
    
    state_frame = analyze_frame( d_prep[len(d_prep)-len_frame-1 : len(d_prep) - 1, 0,9,0] )
    
    
    for i in range(runtime / len_frame):

        if i - 1 % ((runtime/len_frame) / 4) == 0:
            print "    Running frame: " + str(i)
            
            #print q_table
            sys.stdout.flush()

        action_frame = softmax(state_frame, q_table, temperature)
        #print "Action is " + str(action_frame)
        amps_frame = translate_action(action_frame)

        t_frame, d_frame = run_frame(sim_test, stim_test, amps_frame)

        state_frame = analyze_frame(d_frame[:,0,9,0])

        rew_frame, prevlfp = reward_frame(d_frame[:,0,9,0], prevlfp)
        if action_frame != ('non', ):
            rew_frame -= action_weight

        states_all.append(state_frame)
        rewards_all.append(rew_frame)
        actions_all.append(action_frame)
        amps_all = np.concatenate((amps_all, amps_frame), 0)
        d_all = np.concatenate((d_all, d_frame), 0)
        t_all = np.concatenate((t_all, t_frame), 0)

        if i > 0:
            state_t0 = states_all[i-1]
            action_t1 = actions_all[i]
            reward_t1 = rewards_all[i]
            state_t1 = states_all[i]

            tup = (state_t0, action_t1, reward_t1, state_t1)
            tuples_all.append(tup)

            learn_from_tuple(tup, q_table)
    return t_all, d_all, rewards_all, amps_all, actions_all, tuples_all

#Q-Learning

In [7]:
def learn_from_tuple(tup, qtable):
    oldq = qtable.get((tup[0],tup[1]), default)
    
    newq = oldq + learn * (tup[2] + future * best_action(tup[3], qtable)[1] - oldq)
    
    #print "Learning oldq: " + str(oldq) + "newq: " + str(newq)
    
    qtable[(tup[0],tup[1])] = newq

In [8]:
def best_action(state, q_t):
    
    best_act = random.choice(actions)
    best_q = q_t.get((state,best_act), default)
    
    for a in actions:
        if q_t.get((state,a), default) > best_q:
            best_act = a
            best_q = q_t.get((state,a), default)
    
    return best_act, best_q
            

In [9]:
def softmax(state, q_t, tau_s = 0.2):
    
    tops = {}
    probs = []
    
    for a in actions:
        num = np.exp( q_t.get((state,a), default) / tau_s )
        tops[a] = num
        
    denom = 0.0
    for x in tops: denom += float(tops[x])
        
    for a in actions:
        probs.append( tops[a] / denom)
        
    rand = random.random()
    
    p = 0
    for i in range(len(probs)):
        p += probs[i]
        if rand <= p:
            return actions[i]
        
            
    

#State and Reward Stuff

In [10]:
#Assume dt = 1!
def analyze_frame(d_frame, prev_frames = []):
    
    avg = sum(d_frame) / float(len(d_frame))    
    
    if avg < 0:
        state_vector = (-1, )

    else:
        state_vector = (1, )
    
    

    return state_vector

In [11]:
def bp_filter_lfp(raw_data, f_lo, f_hi, f_sampl, order = 5):

        
    b,a = make_butter(f_lo, f_hi, f_sampl, order, 'band')
    data_f = lfilter(b,a,raw_data)
        
    return data_f

def make_butter(f_lo, f_hi, f_sampl, order, f_type):
    f_nyquist = 0.5*f_sampl
    lo = f_lo/f_nyquist
    hi = f_hi/f_nyquist
    if f_type == 'band' or f_type == 'bandpass': 
        b,a = butter(order, [lo, hi], btype = f_type)
    elif f_type == 'lowpass' or f_type == 'low':
        b,a = butter(order, lo, btype = f_type)
    else:
        b,a = butter(order, hi, btype = f_type)
    return b,a

def reward_frame(d_frame, prevlfp, alpha = 0.9, lo = 0.3, hi = 0.5, order = 3):
    
    b,a = make_butter(lo, hi, 1, order, 'band')
    filtered_data = lfilter(b,a,d_frame)
    
    dotted = np.dot (filtered_data, filtered_data)
    lfpbar = (1 - alpha) * prevlfp + dotted * alpha
    rew = -np.log(lfpbar)
    return rew, lfpbar

In [12]:
def translate_action(action):
    
    if (len(action)) > 3:
        balance_charge = action[3]
    action_amps = []
    
    if action[0] == 'sin':
        wave_amp = action[1]
        period = action[2]        
        for i in range(n_frame):
            action_amps.append( wave_amp * np.sin( (2*np.pi / period) * i ))
        if balance_charge:
            action_amps[n_frame - 1] -= sum(action_amps)
    
    elif action[0] == 'sqr':
        wave_amp = action[1]
        period = action[2]
        pulse_width = action[4]
        for i in range(n_frame):
            if (i*dt) % period  == 0:
                amplitude = wave_amp
                
            elif (i*dt - pulse_width/2) % period == 0:
                amplitude = -wave_amp
            
            elif (i*dt - pulse_width) % period == 0:
                amplitude = 0
                
            action_amps.append(amplitude)
       
    elif action[0] == 'non':
        for i in range(n_frame):
            action_amps.append(0)
    

    
    return action_amps

#Epileptor

In [13]:
def run_frame(sim, stim, action_amps):
    for i in range(n_frame):
       
            
            
            
        #params_dict = {'amp' : action_amps[i], 'T' : dt, 'tau' : dt, 'onset' : dt}

        stim.temporal.parameters["amp"] = action_amps[i]
        #stim.temporal.parameters["T"] = runtime
        #stim.temporal.parameters["tau"] = runtime
        
        (t, d), (te, de), (ti, di) = sim.run(simulation_length = dt)
        
        
        if i == 0:
            t_f = copy.deepcopy(t)
            d_f = copy.deepcopy(d)
            
        else: 
            t_f = np.append(t_f, t, axis = 0)
            d_f = np.append(d_f, d, axis = 0)
    
    return t_f, d_f

def prepare_sim(): 
    epileptic = [9]
    almost_epileptic = []
    stimulus_target = [9]
    stimulus_amplitude = 0
    stimulus_weight = 2 #0-2 for each region
    onset = 0
    period = runtime
    pulse_width = runtime
    frame_length = len_frame
    (s, stimulus) = set_sim(epileptic, almost_epileptic, stimulus_target, stimulus_amplitude, 
                  stimulus_weight, onset, period, pulse_width, runtime)


    (t_t, t_d), (aa, aa1), (bb, bb1) = s.run(simulation_length = ic_buffer)

    t_all = copy.copy(t_t)
    d_all = copy.copy(t_d)

    return t_all, d_all, s, stimulus

def set_sim(epileptic, almost_epileptic, target, amp, stim_weight, 
            onset, period, pw, sim_time, K_s=-0.2, K_f=0.1, r_1=0.00015):
  
    #Set up epileptic regions
    epileptors = models.Epileptor(Ks=K_s, Kf=K_f, r=r_1)
    epileptors.x0 = np.ones((76))*-2.4 #set all brain regions non-epileptic
    epileptors.x0[almost_epileptic] = np.ones((2))*-1.9 #overwrite as epileptogenic; borderline epileptic ('close to epilepsy threshold')
    epileptors.x0[epileptic] = np.ones((3))*-1.6 #rHC and lHC are epileptic

    #brain parameters and integration method
    con = connectivity.Connectivity(load_default=True)
    coupl = coupling.Difference(a=1.)
   


    hiss = noise.Additive(nsig = np.array([0., 0., 0., 0.0003, 0.0003, 0.]))
    #heunint = integrators.HeunStochastic(dt=0.05, noise=hiss)
    heunint = integrators.HeunDeterministic(dt=0.05)

    #set up monitors (SEEG and temporal average)
    rm = region_mapping.RegionMapping(load_default=True)
    mon_tavg = monitors.TemporalAverage(period=1.)
    mon_EEG = monitors.EEG(load_default=True,
                       region_mapping=rm,
                       period=1.)
    mon_SEEG = monitors.iEEG(load_default=True,
                             region_mapping=rm,
                             period=1.,
                             ) #intracranial EEG (iEEG) is a particular/specific type of stereoEEG (SEEG)
    what_to_watch = (mon_tavg, mon_EEG, mon_SEEG)
    
    
    #set stimulus to epileptic regions
    weighting = np.ones((76))*0 #np.zeros((76))
    weighting[target] = np.array([stim_weight]) #stimulate rHC and lHC


    #set stimulation paramaeters
    eqn_t = equations.PulseTrain()               #square wave
    eqn_t.parameters["T"] = period             #period
    eqn_t.parameters["onset"] = onset
    eqn_t.parameters["tau"] = pw 
    eqn_t.parameters["amp"] = amp
    stim = patterns.StimuliRegion(temporal = eqn_t,
                          connectivity = con, 
                          weight = weighting)
    
    #Configure space and time
    stim.configure_space()
    stim.configure_time(np.arange(0., sim_time, heunint.dt))

    #Initialise a Simulator -- Model, Connectivity, Integrator, and Monitors.
    sim = simulator.Simulator(model = epileptors, 
                      connectivity = con,
                      coupling = coupl, 
                      integrator = heunint, 
                      monitors = what_to_watch,
                      stimulus = stim,
                      simulation_length = sim_time)
    sim.configure()
    #plot_pattern(stim)
    return (sim, stim)